# Data Clustering & Preprocessing Pipeline
Split cleaned real estate data into rent/sale, normalize prices, cluster by price segment (low/medium/high), and preprocess for modeling.

## 1. Load Libraries and Dataset

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Load the fixed CSV
data_path = "C:\\Users\\user\\OneDrive\\Bureau\\Data Mining Projecy\\Tunisan-Real-Estate-Price-Prediction-Platform\\ML\\data\\processed\\cleaned_real_estate_fixed.csv"

df = pd.read_csv(data_path)

print(f"Dataset loaded: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nData types:\n{df.dtypes}")
print(f"\nMissing values:\n{df.isnull().sum()}")

Dataset loaded: (23510, 15)
Columns: ['price', 'transaction', 'region', 'surface', 'bathrooms', 'rooms', 'property_type', 'has_piscine', 'has_garage', 'has_jardin', 'has_terrasse', 'has_ascenseur', 'is_meuble', 'has_chauffage', 'has_climatisation']

First few rows:
    price transaction    region     surface  bathrooms  rooms property_type  \
0  6800.0        rent  La Marsa  350.000000          1      4   Appartement   
1  5850.0        rent  Carthage   83.250000          3      3         Villa   
2  1500.0        rent  La Marsa  152.117647          2      2   Appartement   
3  5850.0        rent  La Marsa   81.714286          3      2         Villa   
4  2550.0        rent  La Marsa  145.000000          2      2   Appartement   

   has_piscine  has_garage  has_jardin  has_terrasse  has_ascenseur  \
0        False       False       False          True          False   
1         True       False        True         False           True   
2        False        True       False        

## 2. Split Dataset into Rent and Sale

In [7]:
# Identify the transaction type column
print("Unique values in potential type columns:")
for col in df.columns:
    if 'type' in col.lower() or 'transaction' in col.lower() or 'offer' in col.lower():
        print(f"  {col}: {df[col].unique()}")

# Split based on transaction type (prioritize 'transaction' column)
type_col = None
for col in df.columns:
    if col.lower() == 'transaction':
        type_col = col
        break

if type_col is None:
    for col in df.columns:
        if 'type' in col.lower():
            type_col = col
            break

print(f"\nUsing column for split: '{type_col}'")

if type_col:
    print(f"Values in {type_col}: {df[type_col].unique()}")
    rent_df = df[df[type_col].str.lower() == 'rent'].copy()
    sale_df = df[df[type_col].str.lower() == 'sale'].copy()
else:
    print("Error: Could not find transaction type column!")
    rent_df = pd.DataFrame()
    sale_df = pd.DataFrame()

print(f"\nRent listings: {len(rent_df):,}")
print(f"Sale listings: {len(sale_df):,}")
print(f"Total: {len(rent_df) + len(sale_df):,}")

if len(rent_df) > 0:
    print(f"\nRent price stats:")
    print(rent_df['price'].describe())

if len(sale_df) > 0:
    print(f"\nSale price stats:")
    print(sale_df['price'].describe())

Unique values in potential type columns:
  transaction: ['rent' 'sale']
  property_type: ['Appartement' 'Villa' 'Studio' 'Maison']

Using column for split: 'transaction'
Values in transaction: ['rent' 'sale']

Rent listings: 11,781
Sale listings: 11,729
Total: 23,510

Rent price stats:
count    11781.000000
mean      2870.548425
std       2876.125173
min        300.000000
25%       1500.000000
50%       1600.000000
75%       3000.000000
max      20000.000000
Name: price, dtype: float64

Sale price stats:
count    1.172900e+04
mean     6.391396e+05
std      5.473163e+05
min      5.000000e+04
25%      3.000000e+05
50%      4.350000e+05
75%      7.500000e+05
max      5.000000e+06
Name: price, dtype: float64


## 3. Preprocess Features (Encoding + Cleaning)

In [8]:
def preprocess_data(data, dataset_name=""):
    """Preprocess: encode categoricals, select features (no row dropping)"""
    df_proc = data.copy()
    
    # We assume no missing values; keep all rows
    print(f"{dataset_name} - Rows: {len(df_proc):,}")
    
    # Select numeric and categorical features
    numeric_cols = ['price', 'surface', 'rooms', 'bathrooms']
    numeric_cols = [c for c in numeric_cols if c in df_proc.columns]
    
    # Amenity features (binary)
    amenity_cols = ['has_piscine', 'has_garage', 'has_jardin', 'has_terrasse', 'has_ascenseur', 'is_meuble', 'has_chauffage', 'has_climatisation']
    amenity_cols = [c for c in amenity_cols if c in df_proc.columns]
    
    categorical_cols = ['region', 'property_type']
    categorical_cols = [c for c in categorical_cols if c in df_proc.columns]
    
    # Fill missing numeric values with median (if any)
    for col in numeric_cols:
        if df_proc[col].isnull().sum() > 0:
            df_proc[col].fillna(df_proc[col].median(), inplace=True)
    
    # Fill missing amenity values with False (if any)
    for col in amenity_cols:
        if col in df_proc.columns and df_proc[col].isnull().sum() > 0:
            df_proc[col].fillna(False, inplace=True)
    
    # Encode categorical features using LabelEncoder
    le_dict = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df_proc[f'{col}_encoded'] = le.fit_transform(df_proc[col].astype(str))
        le_dict[col] = le
    
    print(f"{dataset_name} - Numeric columns: {numeric_cols}")
    print(f"{dataset_name} - Amenity columns: {amenity_cols}")
    print(f"{dataset_name} - Categorical encoded: {[f'{c}_encoded' for c in categorical_cols]}")
    
    return df_proc, numeric_cols, categorical_cols, amenity_cols, le_dict

# Preprocess both datasets
rent_processed, rent_numeric, rent_cat, rent_amenities, rent_le = preprocess_data(rent_df, "RENT")
sale_processed, sale_numeric, sale_cat, sale_amenities, sale_le = preprocess_data(sale_df, "SALE")

print(f"\nAfter preprocessing:")
print(f"  Rent: {len(rent_processed):,} rows")
print(f"  Sale: {len(sale_processed):,} rows")

RENT - Rows: 11,781
RENT - Numeric columns: ['price', 'surface', 'rooms', 'bathrooms']
RENT - Amenity columns: ['has_piscine', 'has_garage', 'has_jardin', 'has_terrasse', 'has_ascenseur', 'is_meuble', 'has_chauffage', 'has_climatisation']
RENT - Categorical encoded: ['region_encoded', 'property_type_encoded']
SALE - Rows: 11,729
SALE - Numeric columns: ['price', 'surface', 'rooms', 'bathrooms']
SALE - Amenity columns: ['has_piscine', 'has_garage', 'has_jardin', 'has_terrasse', 'has_ascenseur', 'is_meuble', 'has_chauffage', 'has_climatisation']
SALE - Categorical encoded: ['region_encoded', 'property_type_encoded']

After preprocessing:
  Rent: 11,781 rows
  Sale: 11,729 rows


## 4. Normalize Price Column

In [9]:
# Normalize prices separately for rent and sale using StandardScaler
rent_scaler = StandardScaler()
sale_scaler = StandardScaler()

# Apply scaling
rent_processed['price_normalized'] = rent_scaler.fit_transform(rent_processed[['price']])
sale_processed['price_normalized'] = sale_scaler.fit_transform(sale_processed[['price']])

print("Price Normalization (StandardScaler):")
print(f"RENT - Original range: [{rent_processed['price'].min():.0f}, {rent_processed['price'].max():.0f}]")
print(f"RENT - Normalized range: [{rent_processed['price_normalized'].min():.3f}, {rent_processed['price_normalized'].max():.3f}]")
print(f"\nSALE - Original range: [{sale_processed['price'].min():.0f}, {sale_processed['price'].max():.0f}]")
print(f"SALE - Normalized range: [{sale_processed['price_normalized'].min():.3f}, {sale_processed['price_normalized'].max():.3f}]")

Price Normalization (StandardScaler):
RENT - Original range: [300, 20000]
RENT - Normalized range: [-0.894, 5.956]

SALE - Original range: [50000, 5000000]
SALE - Normalized range: [-1.076, 7.968]


## 5. Cluster Rent Listings by Price Segment

In [10]:
def apply_clustering(data, dataset_name="", amenity_cols=None, n_clusters=3):
    """
    Apply KMeans clustering on property features (surface, rooms, bathrooms, amenities).
    Then analyze price segments by region to determine if a location is high/low/medium value.
    """
    df_clust = data.copy()
    
    # Prepare features for clustering: property characteristics EXCEPT price
    clustering_features = ['surface', 'rooms', 'bathrooms']
    clustering_features = [c for c in clustering_features if c in df_clust.columns]
    
    # Add amenity features if available
    if amenity_cols is None:
        amenity_cols = []
    clustering_features.extend([c for c in amenity_cols if c in df_clust.columns])
    
    # Normalize clustering features
    scaler_features = StandardScaler()
    X_cluster_normalized = scaler_features.fit_transform(df_clust[clustering_features])
    
    # Apply KMeans on normalized features (NOT price)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df_clust['property_type_cluster'] = kmeans.fit_predict(X_cluster_normalized)
    
    # Analyze clusters: what do they represent in terms of property characteristics?
    print(f"\n{dataset_name} - Property Type Clusters (based on {', '.join(clustering_features)}):")
    for cluster_id in sorted(df_clust['property_type_cluster'].unique()):
        cluster_data = df_clust[df_clust['property_type_cluster'] == cluster_id]
        print(f"\n  Cluster {cluster_id}:")
        print(f"    Count: {len(cluster_data):,}")
        print(f"    Avg Surface: {cluster_data['surface'].mean():.1f}")
        print(f"    Avg Rooms: {cluster_data['rooms'].mean():.1f}")
        print(f"    Avg Price: {cluster_data['price'].mean():,.0f}")
        print(f"    Price Range: [{cluster_data['price'].min():,.0f}, {cluster_data['price'].max():,.0f}]")
    
    # Now analyze price segments BY REGION to determine if that location is high/low/medium
    print(f"\n{dataset_name} - Analyzing price value by Region...")
    
    # Calculate region market statistics
    region_stats = df_clust.groupby(['region']).agg({
        'price': ['mean', 'median', 'count'],
        'surface': 'mean',
        'rooms': 'mean'
    }).reset_index()
    region_stats.columns = ['region', 'mean_price', 'median_price', 'count', 'avg_surface', 'avg_rooms']
    
    # Sort by mean price to determine thresholds
    all_mean_prices = df_clust.groupby(['region'])['price'].mean()
    price_threshold_low = all_mean_prices.quantile(0.33)
    price_threshold_high = all_mean_prices.quantile(0.67)
    
    # Assign price segment based on region's mean price
    def assign_segment(mean_price):
        if mean_price < price_threshold_low:
            return 'Low'
        elif mean_price < price_threshold_high:
            return 'Medium'
        else:
            return 'High'
    
    region_stats['price_segment'] = region_stats['mean_price'].apply(assign_segment)
    
    # Merge segment back to main dataframe
    df_clust = df_clust.merge(region_stats[['region', 'price_segment']], 
                               on=['region'], how='left')
    
    print(f"\n{dataset_name} - Price Thresholds (by Region mean price):")
    print(f"  Low:    < {price_threshold_low:,.0f}")
    print(f"  Medium: {price_threshold_low:,.0f} - {price_threshold_high:,.0f}")
    print(f"  High:   > {price_threshold_high:,.0f}")
    
    return df_clust, kmeans, scaler_features, region_stats

# Apply clustering to RENT
rent_clustered, rent_kmeans, rent_scaler, rent_stats = apply_clustering(rent_processed, "RENT", amenity_cols=rent_amenities, n_clusters=3)
print("\n" + "="*80)

# Apply clustering to SALE
sale_clustered, sale_kmeans, sale_scaler, sale_stats = apply_clustering(sale_processed, "SALE", amenity_cols=sale_amenities, n_clusters=3)


RENT - Property Type Clusters (based on surface, rooms, bathrooms, has_piscine, has_garage, has_jardin, has_terrasse, has_ascenseur, is_meuble, has_chauffage, has_climatisation):

  Cluster 0:
    Count: 5,717
    Avg Surface: 124.6
    Avg Rooms: 2.1
    Avg Price: 1,886
    Price Range: [300, 20,000]

  Cluster 1:
    Count: 2,020
    Avg Surface: 493.2
    Avg Rooms: 4.3
    Avg Price: 6,938
    Price Range: [550, 20,000]

  Cluster 2:
    Count: 4,044
    Avg Surface: 134.3
    Avg Rooms: 2.2
    Avg Price: 2,231
    Price Range: [450, 15,000]

RENT - Analyzing price value by Region...

RENT - Price Thresholds (by Region mean price):
  Low:    < 1,533
  Medium: 1,533 - 2,130
  High:   > 2,130


SALE - Property Type Clusters (based on surface, rooms, bathrooms, has_piscine, has_garage, has_jardin, has_terrasse, has_ascenseur, is_meuble, has_chauffage, has_climatisation):

  Cluster 0:
    Count: 5,054
    Avg Surface: 198.9
    Avg Rooms: 2.8
    Avg Price: 467,616
    Price Range:

## 6. Summary: Cluster Distribution by City/Region

In [11]:
# Summary of cluster distribution by 
print("=" * 80)
print("RENT - Cluster distribution by Region:")
print("=" * 80)
rent_summary = rent_clustered.groupby(['region', 'price_segment']).size().unstack(fill_value=0)
print(rent_summary)

print("\n" + "=" * 80)
print("SALE - Cluster distribution by Region:")
print("=" * 80)
sale_summary = sale_clustered.groupby(['region', 'price_segment']).size().unstack(fill_value=0)
print(sale_summary)

# Sample of clustered data
print("\n" + "=" * 80)
print("Sample of RENT clustered data:")
print("=" * 80)
print(rent_clustered[['region', 'price', 'price_normalized', 'price_segment']].head(10))

print("\n" + "=" * 80)
print("Sample of SALE clustered data:")
print("=" * 80)
print(sale_clustered[['region', 'price', 'price_normalized', 'price_segment']].head(10))

RENT - Cluster distribution by Region:
price_segment           High  Low  Medium
region                                   
Agba                       0    0       6
Aghir                      5    0       0
Ain Draham                 0    1       0
Ain Zaghouan Nord          0    0     144
Ain Zaghouan Sud           0    0      46
...                      ...  ...     ...
Utique                     1    0       0
Zaghouane Ville            1    0       0
Zaouit Ksibat Thrayett     1    0       0
Zarzis                     0    0       3
Zarzouna                   0    1       0

[221 rows x 3 columns]

SALE - Cluster distribution by Region:
price_segment           High  Low  Medium
region                                   
Agba                       0    0       7
Aghir                     17    0       0
Ain Draham                 0    4       0
Ain Zaghouan Nord          0   54       0
Ain Zaghouan Sud           0   42       0
...                      ...  ...     ...
Utique         

In [12]:
# Build lookup table from training data
rent_lookup = rent_clustered[['region', 'price_segment']].drop_duplicates()
sale_lookup = sale_clustered[['region', 'price_segment']].drop_duplicates()

rent_lookup_path = r"C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan-Real-Estate-Price-Prediction-Platform\ML\data\processed\rent_city_region_segment.csv"
sale_lookup_path = r"C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan-Real-Estate-Price-Prediction-Platform\ML\data\processed\sale_city_region_segment.csv"

rent_lookup.to_csv(rent_lookup_path, index=False)
sale_lookup.to_csv(sale_lookup_path, index=False)

print(f"✅ Rent region segment lookup saved to: {rent_lookup_path}")
print(f"✅ Sale region segment lookup saved to: {sale_lookup_path}")


✅ Rent region segment lookup saved to: C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan-Real-Estate-Price-Prediction-Platform\ML\data\processed\rent_city_region_segment.csv
✅ Sale region segment lookup saved to: C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan-Real-Estate-Price-Prediction-Platform\ML\data\processed\sale_city_region_segment.csv


In [13]:
# Save clustered datasets

rent_output_path = r"C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan-Real-Estate-Price-Prediction-Platform\ML\data\processed\rent_clustered.csv"
sale_output_path = r"C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan-Real-Estate-Price-Prediction-Platform\ML\data\processed\sale_clustered.csv"

# Save, keeping original columns + new ones (only if present)
base_cols = [ 'region', 'price', 'price_normalized', 'surface', 'rooms', 'bathrooms', 'property_type', 'property_type_cluster', 'price_segment',
             'has_piscine', 'has_garage', 'has_jardin', 'has_terrasse', 'has_ascenseur', 'is_meuble', 'has_chauffage', 'has_climatisation']

rent_cols = [c for c in base_cols if c in rent_clustered.columns]
sale_cols = [c for c in base_cols if c in sale_clustered.columns]

rent_save = rent_clustered[rent_cols].copy()
sale_save = sale_clustered[sale_cols].copy()

rent_save.to_csv(rent_output_path, index=False)
sale_save.to_csv(sale_output_path, index=False)

print(f"✅ Rent dataset saved to: {rent_output_path}")
print(f"   - Shape: {rent_save.shape}")
print(f"   - Columns: {list(rent_save.columns)}")

print(f"\n✅ Sale dataset saved to: {sale_output_path}")
print(f"   - Shape: {sale_save.shape}")
print(f"   - Columns: {list(sale_save.columns)}")

# Also save summary statistics
summary_stats = pd.DataFrame({
    'Dataset': ['RENT', 'SALE'],
    'Total_Rows': [len(rent_clustered), len(sale_clustered)],
    'Low_Segment': [
        len(rent_clustered[rent_clustered['price_segment'] == 'Low']),
        len(sale_clustered[sale_clustered['price_segment'] == 'Low'])
    ],
    'Medium_Segment': [
        len(rent_clustered[rent_clustered['price_segment'] == 'Medium']),
        len(sale_clustered[sale_clustered['price_segment'] == 'Medium'])
    ],
    'High_Segment': [
        len(rent_clustered[rent_clustered['price_segment'] == 'High']),
        len(sale_clustered[sale_clustered['price_segment'] == 'High'])
    ],
    'Avg_Price': [
        rent_clustered['price'].mean(),
        sale_clustered['price'].mean()
    ]
})

summary_path = r"C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan-Real-Estate-Price-Prediction-Platform\ML\data\processed\clustering_summary.csv"
summary_stats.to_csv(summary_path, index=False)

print(f"\n✅ Summary statistics saved to: {summary_path}")
print("\nSummary Statistics:")
print(summary_stats.to_string(index=False))

✅ Rent dataset saved to: C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan-Real-Estate-Price-Prediction-Platform\ML\data\processed\rent_clustered.csv
   - Shape: (11781, 17)
   - Columns: ['region', 'price', 'price_normalized', 'surface', 'rooms', 'bathrooms', 'property_type', 'property_type_cluster', 'price_segment', 'has_piscine', 'has_garage', 'has_jardin', 'has_terrasse', 'has_ascenseur', 'is_meuble', 'has_chauffage', 'has_climatisation']

✅ Sale dataset saved to: C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan-Real-Estate-Price-Prediction-Platform\ML\data\processed\sale_clustered.csv
   - Shape: (11729, 17)
   - Columns: ['region', 'price', 'price_normalized', 'surface', 'rooms', 'bathrooms', 'property_type', 'property_type_cluster', 'price_segment', 'has_piscine', 'has_garage', 'has_jardin', 'has_terrasse', 'has_ascenseur', 'is_meuble', 'has_chauffage', 'has_climatisation']

✅ Summary statistics saved to: C:\Users\user\OneDrive\Bureau\Data Mining Projecy\Tunisan